In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np
import pandas as pd
import keras
import tensorflow as tf

file_name = "all_data.csv"

df = pd.read_csv(file_name)
questions = df['question']
answers = df['answer']

Using TensorFlow backend.


In [2]:
tokenizer = keras.preprocessing.text.Tokenizer()

def dataset_preparation(data):

    # basic cleanup
    corpus = data

    # tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

def create_model(predictors, label, max_sequence_len, total_words):

    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences = True))
    # model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
    print( model.summary() )
    return model 

def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [3]:
predictors, label, max_sequence_len, total_words = dataset_preparation(questions)
model = create_model(predictors, label, max_sequence_len, total_words)
# model.save_weights('model_weights.h5')
model.save('model.h5')

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
7226/7226 [==============================] - 15s 2ms/step - loss: 5.4072 - accuracy: 0.0400
Epoch 2/100
  96/7226 [..............................] - ETA: 12s - loss: 5.1983 - accuracy: 0.0104

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


7226/7226 [==============================] - 13s 2ms/step - loss: 5.2036 - accuracy: 0.0450
Epoch 3/100
7226/7226 [==============================] - 12s 2ms/step - loss: 4.9164 - accuracy: 0.0659
Epoch 4/100
7226/7226 [==============================] - 13s 2ms/step - loss: 4.5679 - accuracy: 0.1053
Epoch 5/100
7226/7226 [==============================] - 13s 2ms/step - loss: 4.2695 - accuracy: 0.1560
Epoch 6/100
7226/7226 [==============================] - 13s 2ms/step - loss: 4.0270 - accuracy: 0.1972
Epoch 7/100
7226/7226 [==============================] - 13s 2ms/step - loss: 3.8434 - accuracy: 0.2303
Epoch 8/100
7226/7226 [==============================] - 13s 2ms/step - loss: 3.6764 - accuracy: 0.2634
Epoch 9/100
7226/7226 [==============================] - 13s 2ms/step - loss: 3.5422 - accuracy: 0.2746
Epoch 10/100
7226/7226 [==============================] - 12s 2ms/step - loss: 3.4157 - accuracy: 0.2986
Epoch 11/100
7226/7226 [==============================] - 12s 2ms/step - lo

7226/7226 [==============================] - 13s 2ms/step - loss: 1.0206 - accuracy: 0.7311
Epoch 81/100
7226/7226 [==============================] - 13s 2ms/step - loss: 1.0039 - accuracy: 0.7350
Epoch 82/100
7226/7226 [==============================] - 13s 2ms/step - loss: 0.9887 - accuracy: 0.7412
Epoch 83/100
7226/7226 [==============================] - 13s 2ms/step - loss: 0.9779 - accuracy: 0.7461
Epoch 84/100
7226/7226 [==============================] - 13s 2ms/step - loss: 0.9658 - accuracy: 0.7430
Epoch 85/100
7226/7226 [==============================] - 13s 2ms/step - loss: 0.9557 - accuracy: 0.7473
Epoch 86/100
7226/7226 [==============================] - 14s 2ms/step - loss: 0.9421 - accuracy: 0.7512
Epoch 87/100
7226/7226 [==============================] - 13s 2ms/step - loss: 0.9272 - accuracy: 0.7553
Epoch 88/100
7226/7226 [==============================] - 13s 2ms/step - loss: 0.9178 - accuracy: 0.7566
Epoch 89/100
7226/7226 [==============================] - 13s 2ms/st

In [4]:
output_sequences = []
for line in answers:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        output_sequences.append(n_gram_sequence)
# pad sequences 
max_output_sequences_len = max([len(x) for x in output_sequences])
# print(max_output_sequences_len)

# model = model.load_weights('model_weights.h5')
model = keras.models.load_model('model.h5')
print (generate_text("điểm chuẩn", max_output_sequences_len, max_sequence_len))

điểm chuẩn xét tuyển dựa vào kết quả học tập trung học phổ thông quốc gia gồm những ai không ạ có bị xếp học riêng không ạ có bị thế nào có nhận hơn học phí không ạ có bị xếp học chung trường là bao nhiêu ạ trong tdtu tdtu có giảm không ạ có được thay lập thay ạ có bị xếp lớp học riêng không ạ học phí là bao nhiêu ạ trong quá trình đó là bao nhiêu ạ khu vực 2 có bao nhiêu ngành trong chương trình tiêu chuẩn có trường chương trình chất lượng cao có ngành nào không học tại trường là
